In [1]:
import pandas as pd
import re
import os
from functools import reduce

from hamcrest import *
from constants import (
    UNIVERSITIES,
    UNIVERSITIES_CODES_MAPPING,
    ENROLLMENT_FILES_LIST,
    FUNDING_FILES_LIST,
    BASE_TUITION_FEES,
    GRADUATES_FILES_LIST,
    LEFT_FOR_PART_TIME_STUDIES_FILES_LIST,
    NUMBER_OF_STUDENTS_FILES_LIST,
    EMPLOYEES_FILES_LIST)

In [2]:
universities_lower = [x.lower() for x in UNIVERSITIES]

In [3]:
DATA_PATH = r"C:\Users\iveta\OneDrive\Desktop\magistras\duomenys"
FINANCING_DATA_PATH = DATA_PATH + "/finansavimas"
ENROLLMENT_DATA_PATH = DATA_PATH  + "/istojusiu_skaicius"
PROJECTS_DATA_PATH = DATA_PATH + "/projektai"
BASE_TUITION_FEES_DATA_PATH = DATA_PATH + "/normines_studiju_kainos"
GRADUATES_DATA_PATH = DATA_PATH + "/baigusiu_skaicius"
LEFT_FOR_PART_TIME_STUDIES_DATA_PATH = DATA_PATH + "/isvyke_daliniu_studiju"
NUMBER_OF_STUDENTS_DATA_PATH = DATA_PATH + "/studentu_skaicius"
EMPLOYEES_DATA_PATH = DATA_PATH + "/darbuotoju_skaicius"

In [4]:
def clean_text(text):
    if pd.isna(text):
        return ""
    text = str(text).strip()
    text = text.replace("\xa0", " ")  # remove non-breaking spaces
    text = re.sub(r"\s+", " ", text)  # collapse multiple spaces
    text = text.lower()
    return text

In [5]:
def clean_numbers(text):
    if pd.isna(text):
        return None

    text = str(text).replace(" ", "").replace(",", ".")

    try:
        num = float(text)
        if num.is_integer():
            return int(num)
        return num
    except ValueError:
        return None

In [6]:
def read_excel_data(file_path: str, universities_lower: list, skiprows: int, usecols: list, columnames: list):
    print(f"\nReading file: {file_path}")

    df = pd.read_excel(file_path, skiprows=skiprows, usecols=usecols, names=columnames)

    df['University'] = df['University'].apply(clean_text)


    filtered_data = df[df['University'].isin(universities_lower)].copy()


    missing = set(universities_lower) - set(filtered_data['University'])
    if missing:
        print("Missing universities in this file:", missing)

    filtered_data["University_Code"] = filtered_data["University"].map(UNIVERSITIES_CODES_MAPPING)

    return filtered_data

# Inspect Financing data

In [7]:
all_years = []
for file in FUNDING_FILES_LIST:
    file_path = FINANCING_DATA_PATH + file
    data = read_excel_data(file_path, universities_lower, skiprows=0, usecols=[0, 1], columnames=['University', "Funding"])
    data["Year"] = re.search(r"\d{4}", file).group()  # extract year from filename
    all_years.append(data)

combined_funding = pd.concat(all_years, ignore_index=True)
funding_df = pd.DataFrame(combined_funding).sort_values(["University", "Year"])
assert_that(funding_df.shape[0], equal_to(120))


Reading file: C:\Users\iveta\OneDrive\Desktop\magistras\duomenys/finansavimas/FINANSAVIMAS_2016.xlsx

Reading file: C:\Users\iveta\OneDrive\Desktop\magistras\duomenys/finansavimas/FINANSAVIMAS_2017.xlsx

Reading file: C:\Users\iveta\OneDrive\Desktop\magistras\duomenys/finansavimas/FINANSAVIMAS_2018.xlsx

Reading file: C:\Users\iveta\OneDrive\Desktop\magistras\duomenys/finansavimas/FINANSAVIMAS_2019.xlsx

Reading file: C:\Users\iveta\OneDrive\Desktop\magistras\duomenys/finansavimas/FINANSAVIMAS_2020.xlsx

Reading file: C:\Users\iveta\OneDrive\Desktop\magistras\duomenys/finansavimas/FINANSAVIMAS_2021.xlsx

Reading file: C:\Users\iveta\OneDrive\Desktop\magistras\duomenys/finansavimas/FINANSAVIMAS_2022.xlsx

Reading file: C:\Users\iveta\OneDrive\Desktop\magistras\duomenys/finansavimas/FINANSAVIMAS_2023.xlsx


PermissionError: [Errno 13] Permission denied: 'C:\\Users\\iveta\\OneDrive\\Desktop\\magistras\\duomenys/finansavimas/FINANSAVIMAS_2023.xlsx'

In [66]:
funding_df

,University,Funding,University_Code,Year
2,kauno technologijos universitetas,26 542,KTU,2016
12,kauno technologijos universitetas,21 709,KTU,2017
22,kauno technologijos universitetas,28 679,KTU,2018
38,kauno technologijos universitetas,30 723,KTU,2019
42,kauno technologijos universitetas,33 396,KTU,2020
...,...,...,...,...
71,vytauto didžiojo universitetas,37 899,VDU,2023
81,vytauto didžiojo universitetas,47 769,VDU,2024
91,vytauto didžiojo universitetas,58 413,VDU,2025
101,vytauto didžiojo universitetas,59 695,VDU,2026


# Inspecting standard tuition fees data

In [13]:
all_years = []
for file in BASE_TUITION_FEES:
    file_path = BASE_TUITION_FEES_DATA_PATH + file
    df = pd.read_excel(file_path, skiprows=3, usecols=[2, 4], names=['Bachelors fee', 'Masters fee'])

    year = re.search(r"\d{4}", file).group()
    df['Bachelors fee'] = df['Bachelors fee'].apply(clean_numbers)
    df['Masters fee'] = df['Masters fee'].apply(clean_numbers)

    avg_bach = df['Bachelors fee'].mean()
    avg_mast = df['Masters fee'].mean()

    # Create ONE summary row per year
    summary_row = {
        "Year": year,
        "Average Bachelors fee": round(avg_bach, 2),
        "Average Masters fee": round(avg_mast, 2)
    }

    all_years.append(summary_row)

fees_df = pd.DataFrame(all_years).sort_values(["Year"])

#fees_df = fees_df.drop_duplicates(subset=["Year", "Average Bachelors fee", "Average Masters fee"])
#assert_that(fees_df.shape[0], equal_to(50))

In [14]:
fees_df

,Year,Average Bachelors fee,Average Masters fee
0,2016,4443.60,5437.60
1,2017,4430.97,5424.97
2,2018,4521.30,5531.30
3,2019,5133.63,6275.63
4,2020,5831.33,7114.33
5,2021,6986.00,8347.00
6,2022,7180.40,8572.40
7,2023,7572.60,9002.60
8,2024,7770.20,9200.20
9,2025,8268.20,9698.20


# Inspecting student number

In [26]:
all_years = []
for file in NUMBER_OF_STUDENTS_FILES_LIST:
    file_path = NUMBER_OF_STUDENTS_DATA_PATH + file
    data = read_excel_data(file_path, universities_lower, skiprows=0, usecols=[0, 1], columnames=['University', "Number current students"])
    data["Year"] = re.search(r"\d{4}", file).group()  # extract year from filename
    all_years.append(data)

combined_students = pd.concat(all_years, ignore_index=True)
combined_students_df = pd.DataFrame(combined_students).sort_values(["University", "Year"])
assert_that(combined_students_df.shape[0], equal_to(100))


Reading file: C:\Users\iveta\OneDrive\Desktop\magistras\duomenys/studentu_skaicius/STUDENTU_SKAICIUS_2016.xlsx

Reading file: C:\Users\iveta\OneDrive\Desktop\magistras\duomenys/studentu_skaicius/STUDENTU_SKAICIUS_2017.xlsx

Reading file: C:\Users\iveta\OneDrive\Desktop\magistras\duomenys/studentu_skaicius/STUDENTU_SKAICIUS_2018.xlsx

Reading file: C:\Users\iveta\OneDrive\Desktop\magistras\duomenys/studentu_skaicius/STUDENTU_SKAICIUS_2019.xlsx

Reading file: C:\Users\iveta\OneDrive\Desktop\magistras\duomenys/studentu_skaicius/STUDENTU_SKAICIUS_2020.xlsx

Reading file: C:\Users\iveta\OneDrive\Desktop\magistras\duomenys/studentu_skaicius/STUDENTU_SKAICIUS_2021.xlsx

Reading file: C:\Users\iveta\OneDrive\Desktop\magistras\duomenys/studentu_skaicius/STUDENTU_SKAICIUS_2022.xlsx

Reading file: C:\Users\iveta\OneDrive\Desktop\magistras\duomenys/studentu_skaicius/STUDENTU_SKAICIUS_2023.xlsx

Reading file: C:\Users\iveta\OneDrive\Desktop\magistras\duomenys/studentu_skaicius/STUDENTU_SKAICIUS_20

In [70]:
combined_students_df

,University,Number current students,University_Code,Year
0,kauno technologijos universitetas,10225,KTU,2016
10,kauno technologijos universitetas,9814,KTU,2017
20,kauno technologijos universitetas,9031,KTU,2018
30,kauno technologijos universitetas,8435,KTU,2019
40,kauno technologijos universitetas,8060,KTU,2020
...,...,...,...,...
59,vytauto didžiojo universitetas,8198,VDU,2021
69,vytauto didžiojo universitetas,8210,VDU,2022
79,vytauto didžiojo universitetas,8368,VDU,2023
89,vytauto didžiojo universitetas,8737,VDU,2024


# Inspecting enrolled students

In [27]:
all_years = []
for file in ENROLLMENT_FILES_LIST:
    file_path = ENROLLMENT_DATA_PATH + file
    data = read_excel_data(file_path, universities_lower, skiprows=4, usecols=[0, 1], columnames=['University', "Enrolled Students"])
    data["Year"] = re.search(r"\d{4}", file).group()
    all_years.append(data)

combined_enrolled = pd.concat(all_years, ignore_index=True)
combined_enrolled["Year"] = combined_enrolled["Year"].astype(int)
combined_enrolled["Enrolled Students"] = combined_enrolled["Enrolled Students"].apply(clean_numbers)
combined_enrolled = combined_enrolled.sort_values(by=["University", "Year"])
assert_that(combined_enrolled.shape[0], equal_to(100))


Reading file: C:\Users\iveta\OneDrive\Desktop\magistras\duomenys/istojusiu_skaicius/LAMA_BPO_2016.xlsx

Reading file: C:\Users\iveta\OneDrive\Desktop\magistras\duomenys/istojusiu_skaicius/LAMA_BPO_2017.xlsx

Reading file: C:\Users\iveta\OneDrive\Desktop\magistras\duomenys/istojusiu_skaicius/LAMA_BPO_2018.xlsx

Reading file: C:\Users\iveta\OneDrive\Desktop\magistras\duomenys/istojusiu_skaicius/LAMA_BPO_2019.xlsx

Reading file: C:\Users\iveta\OneDrive\Desktop\magistras\duomenys/istojusiu_skaicius/LAMA_BPO_2020.xlsx

Reading file: C:\Users\iveta\OneDrive\Desktop\magistras\duomenys/istojusiu_skaicius/LAMA_BPO_2021.xlsx

Reading file: C:\Users\iveta\OneDrive\Desktop\magistras\duomenys/istojusiu_skaicius/LAMA_BPO_2022.xlsx

Reading file: C:\Users\iveta\OneDrive\Desktop\magistras\duomenys/istojusiu_skaicius/LAMA_BPO_2023.xlsx

Reading file: C:\Users\iveta\OneDrive\Desktop\magistras\duomenys/istojusiu_skaicius/LAMA_BPO_2024.xlsx

Reading file: C:\Users\iveta\OneDrive\Desktop\magistras\duomeny

In [73]:
combined_enrolled

,University,Enrolled Students,University_Code,Year
0,kauno technologijos universitetas,1546,KTU,2016
10,kauno technologijos universitetas,1596,KTU,2017
20,kauno technologijos universitetas,1332,KTU,2018
30,kauno technologijos universitetas,1180,KTU,2019
40,kauno technologijos universitetas,1150,KTU,2020
...,...,...,...,...
59,vytauto didžiojo universitetas,876,VDU,2021
69,vytauto didžiojo universitetas,843,VDU,2022
79,vytauto didžiojo universitetas,897,VDU,2023
89,vytauto didžiojo universitetas,905,VDU,2024


# Inspecting graduation data

In [28]:
all_universities = []
CODES_TO_UNIVERSITY = {v: k for k, v in UNIVERSITIES_CODES_MAPPING.items()}

for file in GRADUATES_FILES_LIST:
    uni_code = os.path.splitext(file)[0].upper()
    uni_name = CODES_TO_UNIVERSITY.get(uni_code)
    if uni_name is None:
        print(f"Unknown university code in file: {file}")
        continue


    df.columns = df.columns.astype(str).str.strip()
    df.columns = df.columns.map(lambda x: str(x).split('-')[0])

    bachelor = df.loc[df.iloc[:, 0].str.contains("Bakalauras", case=False, na=False)].iloc[0, 1:].values
    master = df.loc[df.iloc[:, 0].str.contains("Magistras", case=False, na=False)].iloc[0, 1:].values
    continious = df.loc[df.iloc[:, 0].str.contains("Vientisosios", case=False, na=False)].iloc[0, 1:].values
    years = df.columns[1:]

    temp = pd.DataFrame({
        "University": uni_name,
        "University_Code": uni_code,
        "Year": years,
        "Bachelor graduates": bachelor,
        "Master graduates": master,
        "Continious graduates": continious,
    })

    all_universities.append(temp)

combined_graduates_df = pd.concat(all_universities, ignore_index=True)

combined_graduates_df["Bachelor graduates"] = (
    combined_graduates_df["Bachelor graduates"].astype(str).str.replace(" ", "").astype(float).astype(int)
)
combined_graduates_df["Master graduates"] = (
    combined_graduates_df["Master graduates"].astype(str).str.replace(" ", "").astype(float).astype(int)
)


In [76]:
combined_graduates_df

,University,University_Code,Year,Bachelor graduates,Master graduates,Continious graduates
0,kauno technologijos universitetas,KTU,2016,1350,970,0
1,kauno technologijos universitetas,KTU,2017,1386,863,0
2,kauno technologijos universitetas,KTU,2018,1365,748,0
3,kauno technologijos universitetas,KTU,2019,1257,761,0
4,kauno technologijos universitetas,KTU,2020,1248,758,13
...,...,...,...,...,...,...
85,vilniaus gedimino technikos universitetas,VGTU,2020,1176,515,65
86,vilniaus gedimino technikos universitetas,VGTU,2021,956,555,75
87,vilniaus gedimino technikos universitetas,VGTU,2022,1126,503,59
88,vilniaus gedimino technikos universitetas,VGTU,2023,910,489,97


# Inspecting foreign students

In [29]:
all_universities = []
CODES_TO_UNIVERSITY = {v: k for k, v in UNIVERSITIES_CODES_MAPPING.items()}

for file in LEFT_FOR_PART_TIME_STUDIES_FILES_LIST:
    uni_code = os.path.splitext(file)[0].upper()
    uni_name = CODES_TO_UNIVERSITY.get(uni_code)
    if uni_name is None:
        print(f"Unknown university code in file: {file}")
        continue

    df = pd.read_excel(os.path.join(LEFT_FOR_PART_TIME_STUDIES_DATA_PATH, file))

    df.columns = df.columns.astype(str).str.strip()
    for col in ["Bakalauro", "Magistro", "Vientisųjų"]:
        df[col] = (
            df[col]
            .astype(str)
            .str.replace(" ", "", regex=False)
            .replace(["", "nan", "None"], "0")
            .astype(float)
            .fillna(0)
            .astype(int)
        )

    df["Year"] = df["Unnamed: 0"].astype(str).str.split("-").str[0].astype(int)

    temp = pd.DataFrame({
        "University": uni_name,
        "University_Code": uni_code,
        "Year": df["Year"],
        "Bachelors exchange students": df["Bakalauro"].astype(int),
        "Master exchange students": df["Magistro"].astype(int),
        "Continuous exchange students": df["Vientisųjų"].astype(int)
    })

    all_universities.append(temp)

combined_exchange_df = pd.concat(all_universities, ignore_index=True)

combined_exchange_df["Bachelors exchange students"] = (
    combined_exchange_df["Bachelors exchange students"].astype(str).str.replace(" ", "").astype(float).astype(int)
)
combined_exchange_df["Master exchange students"] = (
    combined_exchange_df["Master exchange students"].astype(str).str.replace(" ", "").astype(float).astype(int)
)
combined_exchange_df["Continuous exchange students"] = (
    combined_exchange_df["Continuous exchange students"].astype(str).str.replace(" ", "").astype(float).astype(int)
)

In [78]:
combined_exchange_df

,University,University_Code,Year,Bachelors exchange students,Master exchange students,Continuous exchange students
0,kauno technologijos universitetas,KTU,2010,251,63,0
1,kauno technologijos universitetas,KTU,2011,250,50,0
2,kauno technologijos universitetas,KTU,2012,257,59,0
3,kauno technologijos universitetas,KTU,2013,198,61,0
4,kauno technologijos universitetas,KTU,2014,243,106,0
...,...,...,...,...,...,...
147,vilniaus gedimino technikos universitetas,VGTU,2021,127,27,40
148,vilniaus gedimino technikos universitetas,VGTU,2022,163,30,30
149,vilniaus gedimino technikos universitetas,VGTU,2023,213,46,46
150,vilniaus gedimino technikos universitetas,VGTU,2024,326,52,55


# Inspecting employees numbers

In [30]:
all_universities = []
CODES_TO_UNIVERSITY = {v: k for k, v in UNIVERSITIES_CODES_MAPPING.items()}

for file in EMPLOYEES_FILES_LIST:
    uni_code = os.path.splitext(file)[0].upper()
    uni_name = CODES_TO_UNIVERSITY.get(uni_code)
    if uni_name is None:
        print(f"Unknown university code in file: {file}")
        continue

    df = pd.read_excel(os.path.join(EMPLOYEES_DATA_PATH, file))

    df.columns = df.columns.astype(str).str.strip()
    df.columns = df.columns.map(lambda x: str(x).split('-')[0])

    employees = df.loc[df.iloc[:, 0].str.contains("Darbuotojų", case=False, na=False)].iloc[0, 1:].values
    years = df.columns[1:]

    temp = pd.DataFrame({
        "University": uni_name,
        "University_Code": uni_code,
        "Year": years,
        "Number of Employees": employees,
    })

    all_universities.append(temp)

combined_employee_df = pd.concat(all_universities, ignore_index=True)

combined_employee_df["Number of Employees"] = (
    combined_employee_df["Number of Employees"].astype(str).str.replace(" ", "").astype(float).astype(int)
)

In [31]:
combined_employee_df

,University,University_Code,Year,Number of Employees
0,kauno technologijos universitetas,KTU,2016,1137
1,kauno technologijos universitetas,KTU,2017,1083
2,kauno technologijos universitetas,KTU,2018,1012
3,kauno technologijos universitetas,KTU,2019,1002
4,kauno technologijos universitetas,KTU,2020,958
...,...,...,...,...
85,vilniaus gedimino technikos universitetas,VGTU,2020,1357
86,vilniaus gedimino technikos universitetas,VGTU,2021,1353
87,vilniaus gedimino technikos universitetas,VGTU,2022,1315
88,vilniaus gedimino technikos universitetas,VGTU,2023,1294


# Join all dataframes

In [79]:
def merge_dfs_ignore_duplicates(dfs, on, how='outer'):
    def merge_two(df1, df2):
        cols_to_add = [c for c in df2.columns if c not in df1.columns or c in on]
        return df1.merge(df2[cols_to_add], on=on, how=how)

    return reduce(merge_two, dfs)

In [33]:
dataframes = [funding_df, combined_graduates_df, combined_students_df, combined_enrolled, combined_exchange_df, combined_employee_df]
for df in dataframes:
    df['Year'] = df['Year'].astype(str)
df_final = reduce(lambda left, right: pd.merge(left, right, on=['University_Code', "Year", "University"]), dataframes)

In [34]:
df_final

,University,Funding,University_Code,Year,Bachelor graduates,Master graduates,Continious graduates,Number current students,Enrolled Students,Bachelors exchange students,Master exchange students,Continuous exchange students,Number of Employees
0,kauno technologijos universitetas,26 542,KTU,2016,1350,970,0,10225,1546,262,158,0,1137
1,kauno technologijos universitetas,21 709,KTU,2017,1386,863,0,9814,1596,284,100,4,1083
2,kauno technologijos universitetas,28 679,KTU,2018,1365,748,0,9031,1332,253,90,4,1012
3,kauno technologijos universitetas,30 723,KTU,2019,1257,761,0,8435,1180,156,74,8,1002
4,kauno technologijos universitetas,33 396,KTU,2020,1248,758,13,8060,1150,79,48,6,958
...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,vytauto didžiojo universitetas,24 875,VDU,2020,1349,912,62,8609,1031,100,78,1,1766
86,vytauto didžiojo universitetas,26 485,VDU,2021,1021,850,44,8198,876,155,99,1,1738
87,vytauto didžiojo universitetas,32 056,VDU,2022,958,884,32,8210,843,224,67,1,1725
88,vytauto didžiojo universitetas,37 899,VDU,2023,1109,847,50,8368,897,233,107,1,1633


In [35]:
df_final.to_excel('Galutinis.xlsx')

In [15]:
fees_df.to_excel('Average tuition fees.xlsx')